Преобразуем датает, объединеняя новости за одну дату в одну строку

In [ ]:
import nltk
import openpyxl
import pymorphy3
import json
import numpy as np
import pandas as pd
import datetime

In [ ]:
df = pd.read_csv('news_processed.csv')

In [ ]:
df

,title,datetime,news_body
0,Руководители РАО «ЕЭС России» и «Газпрома» Ана...,2000-10-23 13:33,руководитель рао еэс россия газпром анатолий ч...
1,"РАО ""Норильский никель"" (РТС: NKEL, NKELP) в...",2000-10-23 17:08,рао норильский никель ртс nkel nkelp январь се...
2,"ОАО ""Тюменнефтегаз"" довел среднесуточную добыч...",2000-10-24 14:03,тюменнефтегаз довести среднесуточный добыча не...
3,При объединении операторов связи Сибирского ре...,2000-10-24 18:21,объединение оператор связь сибирский регион до...
4,Генеральный директор ОАО «Электросвязь» Кемеро...,2000-10-24 18:22,генеральный директор электросвязь кемеровский ...
...,...,...,...
203264,Московская биржа обновит методику расчета валю...,2023-04-07 19:04,московский биржа планировать обновить методика...
203265,"Согласно второй оценке Росстата, спад экономик...",2023-04-07 19:22,росстат подтвердить снижение экономика россия ...
203266,Мосбиржа намерена вновь увеличить состав набсо...,2023-04-07 20:08,московский биржа намерить вновь увеличить сост...
203267,Нефть и дивидендные надежды поддерживают рост ...,2023-04-07 20:57,российский фондовый рынок закрыть рост четвёрт...


In [ ]:
def getDateOnly(date):
  return datetime.date(date.year, date.month, date.day)

In [ ]:
def getDayNews(df):
  result = pd.DataFrame()
  prev_date = getDateOnly(datetime.datetime.strptime(df.iloc[0]['datetime'], '%Y-%m-%d %H:%M'))
  day_news = ''

  n = len(df)
  for i in range(n):
    
    curr_date = getDateOnly(datetime.datetime.strptime(df.iloc[i]['datetime'], '%Y-%m-%d %H:%M'))
    if (prev_date == curr_date):
      day_news += str(df.iloc[i]['news_body']) + " "
    else: 
      result = pd.concat([result, pd.DataFrame([{'date':curr_date, 'news_body': day_news}])])
      prev_date = curr_date
      day_news = str(df.iloc[i]['news_body']) + " "
  result = result.reset_index(drop=True)
  return result

In [ ]:
df.iloc[4]['news_body']

'генеральный директор электросвязь кемеровский область виктор жигулин считать хороший вариант объединение сохранение региональный монополист статус юридический лицо заявить презентация посвятить включение акция компания листинг фондовый биржа ртс противный случай считать руководитель предприятие нанести ущерб бюджет регион добавить субъект федерация лишиться крупный налогоплательщик наш информация предложение высказываться впервые отрасль уровень генеральный директор предприятие возможно учесть консультант разработка план объединение компания ик финанс аналитик'

In [ ]:
df_by_days = getDayNews(df)

In [ ]:
df_by_days

,date,news_body
0,2000-10-24,руководитель рао еэс россия газпром анатолий ч...
1,2000-10-25,тюменнефтегаз довести среднесуточный добыча не...
2,2000-10-26,electricite de france заявить намерение инвест...
3,2000-10-27,тулэнерго сохранять ограничение поставка элект...
4,2000-10-30,bp amoco предварительный переговоры роснефть п...
...,...,...
5858,2023-04-03,апр риа новость власть шри ланка планировать п...
5859,2023-04-04,котировка азиатский индекс понедельник утром т...
5860,2023-04-05,вторник апрель мск фьючерс американский индекс...
5861,2023-04-06,среда апрель мск фьючерс американский индекс s...


сохранение новостей объединенных по дням

In [ ]:
df_by_days.to_csv('news_by_days.csv',index=False)

## Собираем датасет за несколько предыдущих дней

In [ ]:
df_by_days = pd.read_csv('news_by_days.csv')

In [ ]:
def getCummulative(df, days):
  result = pd.DataFrame()
  n =  len(df)
  cummulative = list(df['news_body'][:days])

  prev_date = getDateOnly(datetime.datetime.strptime(df.iloc[days-1]['date'], '%Y-%m-%d'))
  for i in range(days, n):
    curr_date = getDateOnly(datetime.datetime.strptime(df.iloc[i]['date'], '%Y-%m-%d'))

    # если даты пропущены то берем, что есть
    if (curr_date - prev_date).days > 1:
      temp_date = prev_date
      while (curr_date - temp_date).days > 1:
        temp_date += datetime.timedelta(days=1)
        result = pd.concat([result, pd.DataFrame([{'date':temp_date, 'news_body': ''.join(cummulative)}])])

    result = pd.concat([result, pd.DataFrame([{'date':curr_date, 'news_body': ''.join(cummulative)}])])
    
    # убираем первый элемент и добавляем в конец текущий
    cummulative.pop(0)
    cummulative.append(df.iloc[i]['news_body'])
    prev_date = curr_date

  result = result.reset_index(drop=True)
  return result

сохранение новостей с несколькими предыдущими датами, если даты пропущены, то берутся значение окна с предыдущей даты

In [ ]:
for i in [1,3,5,7,10,14]:
  getCummulative(df_by_days, i).to_csv(f'news_by_days_{i}.csv', index=False)

In [ ]:
df_result = pd.read_csv('news_by_days_1.csv')

In [ ]:
def getDateDiffs(dates):
  dates.values.tolist()
  result = []
  for i in range(1, len(dates)):
    result.append((getDateOnly(dates[i]) - getDateOnly(dates[i-1])).days)
  df = pd.DataFrame(result, columns=['DATE_DIFF'])
  return df.groupby(['DATE_DIFF'])['DATE_DIFF'].count()

In [ ]:
getDateDiffs(df_result['date'].apply(lambda x: getDateOnly(datetime.datetime.strptime(x, '%Y-%m-%d')) ))

DATE_DIFF
1    8199
Name: DATE_DIFF, dtype: int64

In [ ]:
df_result.iloc[2500:2510]

,date,news_body
2500,2007-08-30,цб рф обнародовать дать остаток средство корсч...
2501,2007-08-31,цб рф остаток средство корсчёт остаток средств...
2502,2007-09-01,коминбанка отозвать лицензия осуществление бан...
2503,2007-09-02,коминбанка отозвать лицензия осуществление бан...
2504,2007-09-03,коминбанка отозвать лицензия осуществление бан...
2505,2007-09-04,акционер ммк утвердить дивиденд размер рубль а...
2506,2007-09-05,цб рф дать остаток корсчёт остаток средство кр...
2507,2007-09-06,цб рф дать остаток корсчёт остаток средство кр...
2508,2007-09-07,цб рф дать остаток корсчёт остаток средство кр...
2509,2007-09-08,цб рф дать остаток корсчёт остаток средство кр...
